In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # remove when finished
from implementations import *
from tools import *
from helpers import *

In [2]:
# from prepare import *
# from costs import *
# from grid_search import *
# from gradient_descent import *
# from stochastic_gradient_descent import *
# from build_polynomial import *
# from least_squares import *
# from split_data import *
# from ridge_regression import * 
# from proj1_helpers import *

In [3]:
# label, features, ids = load_csv_data(data_path, sub_sample=True)

In [4]:
data_tr = pd.read_csv('data/train.csv')
data_te = pd.read_csv('data/test.csv')
idx_tr = data_tr.Id
idx_te = data_te.Id
# data_tr.head()

In [ ]:
features_names = data_tr.columns.drop(['Id','Prediction'])
# features_names

In [ ]:
def hist_for_feature(data,dataCol):
    data1 = data.loc[(data[dataCol]!=-999)].copy()
    data1_s = data1.loc[(data.Prediction=='s')].copy()
    data1_b = data1.loc[(data.Prediction=='b')].copy()

    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(20, 8))
    hist_range = (min(data1_s[dataCol]),max(data1_s[dataCol]))
    ax1.hist(data1_s[dataCol],log=False,range = hist_range,color = '#e0f3db')
    ax1.set_title(dataCol+'_s')
    ax2.hist(data1_b[dataCol],log=False,range = hist_range, color = '#999999')
    ax2.set_title(dataCol+'_b')
    plt.savefig('image/hist_notlog/'+dataCol+'.jpg')
    del ax1,ax2
    plt.close('all')

In [ ]:
# for feature in features:
#     hist_for_feature(data_tr,feature)

In [ ]:
def missing_rate(data):
    features = data.columns.drop(['Id','Prediction'])
    df_ = pd.DataFrame({'idx':[np.nan]})
    for feature in features:
        data2 = data.loc[(data[feature]==-999)].copy()
        missing_rate = data2[feature].size/data[feature].size
        df1 = pd.DataFrame({feature:[missing_rate]})
        df_ = pd.concat([df_,df1],axis=1)
    df_ = df_.drop(columns='idx')
    df_ = df_.T.rename(columns={0:'rate'})
    return df_

In [ ]:
def drop_columns(df_miss):
    names_70 = list(df_miss.loc[df_miss.rate==0.709828].index)
    names_30 = list(df_miss.loc[df_miss.rate==0.399652].index)
    names_phi = list(['PRI_jet_leading_phi','PRI_jet_subleading_phi','PRI_lep_phi','PRI_met_phi','PRI_tau_phi','PRI_jet_num'])
    corr0 = list(['DER_mass_MMC','DER_pt_tot','PRI_tau_pt','PRI_lep_pt'])
    corr1_2 = list(['DER_mass_MMC','DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet','PRI_jet_all_pt','PRI_lep_pt'])
    df_drop_name = {'names_70':names_70,'names_30':names_30,'names_phi':names_phi,'corr0':corr0,'corr1_2':corr1_2}

    return df_drop_name

In [ ]:
df_miss_tr = missing_rate(data_tr)
df_miss_tr

In [5]:
# replacing the -999 with mode in signal or background

DER_mode_s = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].value_counts().index[0]#value_counts().index[0]
DER_mode_b = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].value_counts().index[0]#mean(axis=0)

idx_s = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].index
idx_b = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].index

data_tr.loc[idx_s,'DER_mass_MMC']=DER_mode_s
data_tr.loc[idx_b,'DER_mass_MMC']=DER_mode_b


In [6]:
DER_mode_s

119.89

In [7]:
DER_mode_b

96.415

In [9]:
data_te

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,350000,?,-999.000,79.589,23.916,3.036,-999.000,-999.000,-999.000,0.903,...,2.022,98.556,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
1,350001,?,106.398,67.490,87.949,49.994,-999.000,-999.000,-999.000,2.048,...,-1.138,176.251,1,47.575,-0.553,-0.849,-999.000,-999.000,-999.000,47.575
2,350002,?,117.794,56.226,96.358,4.137,-999.000,-999.000,-999.000,2.755,...,-1.868,111.505,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
3,350003,?,135.861,30.604,97.288,9.104,-999.000,-999.000,-999.000,2.811,...,1.172,164.707,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
4,350004,?,74.159,82.772,58.731,89.646,1.347,536.663,-0.339,1.028,...,-0.231,869.614,3,254.085,-1.013,-0.334,185.857,0.335,2.587,599.213
5,350005,?,95.709,94.168,66.280,14.719,-999.000,-999.000,-999.000,3.065,...,2.566,86.129,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
6,350006,?,85.798,49.059,66.131,37.074,-999.000,-999.000,-999.000,2.253,...,3.060,98.461,1,36.990,-3.277,1.369,-999.000,-999.000,-999.000,36.990
7,350007,?,429.273,75.057,234.610,71.019,0.590,174.261,2.278,4.287,...,1.722,274.193,2,119.440,1.243,-0.739,59.619,1.833,2.788,179.059
8,350008,?,70.958,66.329,60.950,0.758,-999.000,-999.000,-999.000,2.370,...,-1.389,102.088,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
9,350009,?,110.539,2.820,62.136,179.753,3.256,337.201,-2.059,1.092,...,-0.794,361.566,2,115.867,-0.859,1.961,40.427,2.397,1.479,156.294


In [11]:
data_te.loc[(data_te['DER_mass_MMC']!=-999)]['DER_mass_MMC'].value_counts()#value_counts().index[0]

96.728     18
110.070    17
93.123     17
113.899    17
94.545     16
93.050     16
112.171    16
109.983    16
84.671     16
103.139    16
113.705    16
96.356     15
105.093    15
122.974    15
121.401    15
102.021    15
103.495    15
91.300     15
97.264     15
120.551    15
111.452    15
108.238    15
122.967    15
102.666    15
96.994     15
130.734    15
120.817    15
96.135     15
130.895    15
120.257    15
           ..
61.187      1
202.783     1
251.305     1
154.239     1
216.986     1
52.796      1
27.549      1
161.992     1
307.659     1
214.540     1
252.933     1
378.976     1
405.441     1
148.532     1
47.232      1
332.394     1
182.005     1
153.978     1
169.057     1
158.465     1
170.264     1
143.882     1
64.759      1
48.903      1
45.631      1
196.255     1
199.198     1
50.958      1
63.696      1
64.000      1
Name: DER_mass_MMC, Length: 154709, dtype: int64

In [13]:
# replacing the -999 with mode in signal or background

DER_mode_s = data_te.loc[(data_te['DER_mass_MMC']!=-999)].loc[(data_te['Prediction'])=='s']['DER_mass_MMC'].value_counts().index[0]#value_counts().index[0]
DER_mode_b = data_te.loc[(data_te['DER_mass_MMC']!=-999)].loc[(data_te['Prediction'])=='b']['DER_mass_MMC'].value_counts().index[0]#mean(axis=0)

idx_s = data_te.loc[(data_te['DER_mass_MMC']==-999)].loc[(data_te['Prediction'])=='s']['DER_mass_MMC'].index
idx_b = data_te.loc[(data_te['DER_mass_MMC']==-999)].loc[(data_te['Prediction'])=='b']['DER_mass_MMC'].index

data_te.loc[idx_s,'DER_mass_MMC']=DER_mode_s
data_te.loc[idx_b,'DER_mass_MMC']=DER_mode_b


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
DER_mode_s

In [ ]:
DER_mode_te = data_te.loc[(data_te['DER_mass_MMC']!=-999)]['DER_mass_MMC'].value_counts().index[0]#mean(axis=0)
idx_te = data_te.loc[(data_te['DER_mass_MMC']==-999)]['DER_mass_MMC'].index
data_te.loc[idx_te,'DER_mass_MMC']=DER_mode_te

In [ ]:
drop_col = drop_columns(df_miss_tr)
drop_col

'DER_deltaeta_jet_jet','DER_mass_jet_jet','DER_prodeta_jet_jet','DER_lep_eta_centrality','PRI_jet_subleading_pt','PRI_jet_subleading_eta','PRI_jet_subleading_phi': missing rate = 70.9% (PRI_jet_num <= 1)

'PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi': missing rate = 39.9% (PRI_jet_num = 0)

In [ ]:
def group_features_by_jet(df):
    return {  
        0: df.loc[df['PRI_jet_num'] == 0].copy(),
        1: df.loc[df['PRI_jet_num'] == 1].copy(),
        2: df.loc[(df['PRI_jet_num'] == 2) | (df['PRI_jet_num'] == 3)].copy(),
    }

In [ ]:
feature_tr_grp = group_features_by_jet(data_tr)
feature_te_grp = group_features_by_jet(data_te)

In [ ]:
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['names_30']+drop_col['corr0'])))
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = 'PRI_jet_all_pt')# all 0s
feature_tr_grp[1] = feature_tr_grp[1].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['corr1_2'])))
feature_tr_grp[2] = feature_tr_grp[2].drop(columns = list(set(drop_col['names_phi']+drop_col['corr1_2'])))

feature_te_grp[0] = feature_te_grp[0].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['names_30']+drop_col['corr0'])))
feature_te_grp[0] = feature_te_grp[0].drop(columns = 'PRI_jet_all_pt')# all 0s
feature_te_grp[1] = feature_te_grp[1].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['corr1_2'])))
feature_te_grp[2] = feature_te_grp[2].drop(columns = list(set(drop_col['names_phi']+drop_col['corr1_2'])))

In [ ]:
# ridge regression
# w = []
# acc_tr = []
# acc_te = []
# lambda_ = 0.0001
# k_fold = 10
# seed = 1
# y_te = []

# for i in range(len(feature_tr_grp)):
#     x = feature_tr_grp[i].drop(columns = ['Id','Prediction']).values
#     x = log_process(x)
#     x = build_polynomial_features(x,10)
#     x_tr,_,_ = standardize(x)
#     # x_tr = np.hstack((np.ones((x_tr.shape[0], 1)), x_tr)) # have been added in cross validation
#     y_tr = feature_tr_grp[i].Prediction.replace(['s','b'],[1,-1]).values
#     w_tmp,acc_tr_tmp,acc_te_tmp = cv_loop(y_tr, x_tr,k_fold,seed,ridge_regression,lambda_=lambda_)
#     w.append(w_tmp)
#     acc_tr.append(acc_tr_tmp)
#     acc_te.append(acc_te_tmp)
    
#     x = feature_te_grp[i].drop(columns = ['Id','Prediction']).values
#     x = log_process(x)
#     x = build_polynomial_features(x,10)
#     x_te,_,_ = standardize(x)
#     # x_te = np.hstack((np.ones((x_te.shape[0], 1)), x_te))
#     y_te.append(predict_labels(w[i], x_te))

In [ ]:
# logistics
w = []
acc_tr = []
y_te = []
max_iters = 3000

for i in range(len(feature_tr_grp)):
    x = feature_tr_grp[i].drop(columns = ['Id','Prediction']).values
    x = log_process(x)
    x = build_polynomial_features(x,2)
    x_tr,_,_ = standardize(x)
    y_tr = feature_tr_grp[i].Prediction.replace(['s','b'],[1,-1]).values

    loss,w_tmp = logistic_regression_GD(y_tr, x_tr, max_iters)
    y_tr = y_tr.reshape(len(y_tr),1)
    y_tr_pred = predict_labels(w_tmp, x_tr)
    acc_tr_tmp = compute_accuracy(y_tr_pred,y_tr)
    w.append(w_tmp)
    acc_tr.append(acc_tr_tmp) 
    
    x = feature_te_grp[i].drop(columns = ['Id','Prediction']).values
    x = log_process(x)
    x = build_polynomial_features(x,2)
    x_te,_,_ = standardize(x)
    y_te.append(predict_labels(w[i], x_te))    


In [ ]:
acc_tr

In [12]:
y_pred_te = np.concatenate((y_te[0],y_te[1],y_te[2]))
idx_te = np.concatenate((feature_te_grp[0].Id.values,feature_te_grp[1].Id.values,feature_te_grp[2].Id.values))

NameError: name 'y_te' is not defined

In [ ]:
create_csv_submission(idx_te, y_pred_te, 'submission_logistics.csv')

In [ ]:
tx = x_tr
eignvalue,_ = np.linalg.eig(tx.T.dot(tx))
gamma = 1/(0.5*eignvalue.max())*10

In [ ]:
gamma